<table width="100%" border="3">
  <tbody>
    <tr>
      <td width=120><img src="https://is4-ssl.mzstatic.com/image/thumb/Purple128/v4/9f/38/cb/9f38cb51-c606-6fd4-54b3-47bfdb06b712/source/256x256bb.jpg" alt="Aidea" width="120"/></td>
      <td align='left'><h1>TensorFlow 2 Object Detection</h1><BR><h2>D3-MBSH Baseline Solution</h2></td>
    </tr>
  </tbody>
</table>

# Baseline Solution for Mosquito Breeding Site Hunting 

__IEEE ICIP 2019 Grand Challenge__ https://aidea-web.tw/icip



## Step 1: Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

## Step 5: Install packages

In [ ]:
!pip install tf_slim
!pip install lvis
!pip install tensorflow-addons

## Step 6: Set up the working environment

In [ ]:
%env PYTHONPATH=/env/python:/content/drive/My Drive/models:/content/drive/My Drive/models/research:/content/drive/My Drive/models/research/slim

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/models')

In [ ]:
from os import chdir
chdir("/content/drive/My Drive/models/research/")

## Step 7: Test Object Detection API

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

# Part 2: Model prediction

## Step 1: Download the model from the teacher’s cloud drive or use a model trained by yourself

https://drive.google.com/file/d/1Ihqblvz8oCHuHy3hLF9LqeD-mYdPvsGK/view?usp=sharing

https://drive.google.com/file/d/1wv7lNLdYsOADZvCkCH6jzvHYdX9v3FdQ/view?usp=sharing

In [ ]:
!gdown --id 1Ihqblvz8oCHuHy3hLF9LqeD-mYdPvsGK

In [ ]:
!gdown --id 1wv7lNLdYsOADZvCkCH6jzvHYdX9v3FdQ

In [ ]:
!unzip efficientdet_d0_mbsh.zip

In [ ]:
!ls efficientdet_d0_mbsh/checkpoint

## Step 2: Setup

In [ ]:
import tensorflow as tf

import matplotlib
import matplotlib.pyplot as plt

import io
import os
import pathlib
import random
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [ ]:
def load_image_into_numpy_array(path):
  #Load an image from file into a numpy array.
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  image = image.convert("RGB")
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def get_keypoint_tuples(eval_config):
  """Return a tuple list of keypoint edges from the eval config.
  
  Args:
    eval_config: an eval config containing the keypoint edges
  
  Returns:
    a list of edge tuples, each in the format (start, end)
  """
  tuple_list = []
  kp_list = eval_config.keypoint_edge
  for edge in kp_list:
    tuple_list.append((edge.start, edge.end))
  return tuple_list

## Step 3: Model configuration

In [ ]:
import os
pipeline_path = 'efficientdet_d0_mbsh/mbsh_pipeline.config'
model_dir = 'efficientdet_d0_mbsh/checkpoint'
configs = config_util.get_configs_from_pipeline_file(pipeline_path)

In [ ]:
def get_model_detection_function(model):
  """Get a tf.function for detection."""
  @tf.function
  def detect_fn(image):
    """Detect objects in image."""
    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)
    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

In [ ]:
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)
detect_fn = get_model_detection_function(detection_model)

In [ ]:
# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(model_dir, 'ckpt-301')).expect_partial()

In [ ]:
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=False)

## Step 3: Use the trained model to predict objects in the test set

In [ ]:
import numpy as np
from PIL import Image
from IPython.display import display

In [ ]:
image_dir = 'data/test_cdc/test_images/'
image_name = random.choice(os.listdir(image_dir))
print(image_name)
image_path = image_dir + image_name
test_image = Image.open(image_path)
display(test_image)

In [ ]:
image_np = load_image_into_numpy_array(image_path)
input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)
width  = image_np.shape[1]
height = image_np.shape[0]
print(f"{width}x{height}")

In [ ]:
detections, predictions_dict, shapes = detect_fn(input_tensor)
print(detections["detection_scores"])

In [ ]:
number=3

In [ ]:
label_id_offset = 1
image_np_with_detections = image_np.copy()
# Use keypoints if available in detections
keypoints, keypoint_scores = None, None
if 'detection_keypoints' in detections:
  keypoints = detections['detection_keypoints'][0].numpy()
  keypoint_scores = detections['detection_keypoint_scores'][0].numpy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=number,
      min_score_thresh=.01,
      agnostic_mode=False,
      keypoints=keypoints,
      keypoint_scores=keypoint_scores,
      keypoint_edges=get_keypoint_tuples(configs['eval_config']))

plt.figure(figsize=(12,16))
plt.imshow(image_np_with_detections)
plt.show()

# Part 3: Perform object detection in the test set

## Step 1: Perform object detection

In [ ]:
import os
import glob
import pandas as pd
Threshold = 0.3
classname = ['Nan', 'aquarium', 'bottle', 'bowl', 'box', 'bucket', 'plastic_bag', 
             'plate', 'styrofoam', 'tire', 'toilet', 'tub', 'washing_machine',
             'water_tower']
label_id_offset = 1
files = sorted(glob.glob('data/test_cdc/test_images/*.jpg'))
blist = []
for img_file in files:
  fname = os.path.basename(img_file)
  print(f"file:{fname} ", end="")
  image_np = load_image_into_numpy_array(img_file)
  width  = image_np.shape[1]
  height = image_np.shape[0]


  input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
  detections, predictions_dict, shapes = detect_fn(input_tensor)

  scores  =detections['detection_scores'][0].numpy()
  label_ids=detections['detection_classes'][0].numpy().astype(int) + label_id_offset
  boxes    =detections['detection_boxes'][0].numpy()

  bcount = 0;
  for score in list(scores):
    if score < Threshold:
      break
    bcount += 1
  if bcount==0 and scores[0] >0.2:
    bcount = 1
  print(f" count={bcount}")
  
  for i in range(bcount):
    score = scores[i]
    label_id = label_ids[i].astype(int)
    label_name = classname[label_id]
    box = boxes[i]
    xmin = round(box[0]*width)
    xmax = round(box[2]*width)
    ymin = round(box[1]*height)
    ymax = round(box[3]*height)
    record = (fname, label_id, xmin, ymin, xmax-xmin, ymax-ymin, score)
    blist.append(record)


## Step 2: Output detection result

In [ ]:
column_name = ['image_filename', 'label_id', 'x', 'y', 'w', 'h', 'confidence']
df = pd.DataFrame(blist, columns=column_name)
df.to_csv(('mbsh/test_labels_hat2.csv'), index=None)